<a href="https://colab.research.google.com/github/yuriyasui/PDF-RAG/blob/main/Almondo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OPENAI_KEY
OPENAI_API_KEY = "" #@param {type:"string"}

In [ ]:
Question = "On which datasets does GPT-3 struggle?" # "What's the differences between zero-shot and one-shot?"
PDF = 'https://arxiv.org/pdf/2005.14165.pdf'

In [ ]:
# Install necessary module
# print("Installing ....")
!pip install langchain openai pypdf faiss-gpu tiktoken --quiet
# print("Success!!!!!!")

In [ ]:
# Document loading
# from langchain.document_loaders import PyPDFLoader

# file = 'https://arxiv.org/pdf/2005.14165.pdf'
# loader = PyPDFLoader(file)
# document = loader.load()

# from langchain.document_loaders import PagedPDFSplitter

# file = 'https://arxiv.org/pdf/2005.14165.pdf'
# document = PagedPDFSplitter(file)
# pages = document.load_and_split()

# for i in range(10, 15):
#   print('Page number:', i)
#   print(pages[i].page_content)
#   print(' ')

In [ ]:
# Document loading

from langchain.document_loaders import PyPDFLoader

file = PDF
loader = PyPDFLoader(file)
document = loader.load()

In [ ]:
# split document into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
pages = text_splitter.split_documents(document)

# for i in range(10, 15):
#   print('Page number:', i)
#   print(pages[i].page_content)
#   print(' ')


In [ ]:
# Vectorize

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
db = FAISS.from_documents(pages, embeddings)

In [ ]:
# Retriever

query = Question
retriever = db.as_retriever(search_kwargs={"k": 5})
results = retriever.get_relevant_documents(query)

for result in results:
    print(str(result.metadata["page"]) + ":", result.page_content)

0: achieves strong performance on many NLP datasets, including translation, question-answering, and
cloze tasks, as well as several tasks that require on-the-ﬂy reasoning or domain adaptation, such as
unscrambling words, using a novel word in a sentence, or performing 3-digit arithmetic. At the same
time, we also identify some datasets where GPT-3’s few-shot learning still struggles, as well as some
datasets where GPT-3 faces methodological issues related to training on large web corpora. Finally,
we ﬁnd that GPT-3 can generate samples of news articles which human evaluators have difﬁculty
distinguishing from articles written by humans. We discuss broader societal impacts of this ﬁnding
and of GPT-3 in general.
∗Equal contribution
†Johns Hopkins University, OpenAI
Author contributions listed at end of paper.arXiv:2005.14165v4  [cs.CL]  22 Jul 2020
32: predecessor GPT-2, it still has notable weaknesses in text synthesis and several NLP tasks. On text synthesis, although
the overall qual

In [ ]:
# Prompt Template

from langchain_core.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot that responds to a question based on the knowledge provided by me."), #AI: need to improve the prompt
        ("human", "Using the provided information: {text}, please answer the following question: {question}. Provide answer with some examples and the reasoning")
    ]
)

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=OPENAI_API_KEY)

In [ ]:
# Chaining

chain = chat_template | llm
answer = chain.invoke(
    {
        "text": results,
        "question": Question,
    }
)
print(answer.content)

GPT-3 struggles on certain datasets despite its strong performance on many NLP tasks. Some examples of datasets where GPT-3 faces challenges include:

1. ANLI dataset: GPT-3 struggles with natural language inference tasks on this dataset.
2. RACE dataset: GPT-3 faces difficulties with some reading comprehension datasets like RACE.
3. QuAC dataset: GPT-3 performs poorly on QuAC, a dataset that requires modeling structured dialog acts and answer span selections of teacher-student interactions.

These limitations highlight areas where GPT-3's few-shot learning capabilities are not as effective, emphasizing the need for further research and improvement in these specific domains.
